Please think about 2 more analytics, which provide insights for existing/future ​Business Owners​, 
to make important decisions regarding new business or business expansion.

# Role Model

# Find Most Common Attributes of Restaurants that are best in any of the categories that your restaurant competes in the same city

## Set Up


In [1]:
from pymongo import MongoClient
import pandas as pd
import pprint
import geopy.distance
import numpy as np
import json


client = MongoClient()
dblist = client.list_database_names()
db = client.yelp
pp = pprint.PrettyPrinter(indent=4)

## Find Your Restaurant

In [2]:
my_business = db.business.find_one({"name": "MeNami"})
my_business

{'_id': ObjectId('5dc9d09d2c31ec23f5a6ce1e'),
 'business_id': 'C8_zdU7zGLUK3uC4e5AepQ',
 'name': 'MeNami',
 'address': '5469 Yonge Street',
 'city': 'Toronto',
 'state': 'ON',
 'postal_code': 'M2N 5S1',
 'latitude': 43.7778404,
 'longitude': -79.4147699,
 'stars': 3.5,
 'review_count': 371,
 'is_open': 1,
 'attributes': {'GoodForDancing': 'False',
  'HasTV': 'True',
  'CoatCheck': 'False',
  'RestaurantsPriceRange2': '2',
  'Smoking': "u'no'",
  'RestaurantsGoodForGroups': 'True',
  'GoodForKids': 'False',
  'RestaurantsTakeOut': 'True',
  'BikeParking': 'True',
  'RestaurantsDelivery': 'False',
  'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}",
  'RestaurantsReservations': 'True',
  'GoodForMeal': "{'dessert': False, 'latenight': True, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False

## Find Similar Restaurants

In [3]:
query = {
    "city": my_business["city"],
    "state": my_business["state"]
}
project = {
    
}

In [4]:
cursor = db.business.find(query)

In [5]:
lst_business = list(cursor)

In [6]:
df_business = pd.DataFrame(lst_business)
df_business = df_business[df_business['is_open'] == 1]
df_business = df_business[df_business['review_count'] > 20]
df_business = df_business[df_business['stars'] > my_business['stars']]
df_business = df_business.drop(['city', 'state', 'business_id', '_id', 'is_open', 'hours'], axis=1)

In [7]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1853 entries, 0 to 18902
Data columns (total 9 columns):
name            1853 non-null object
address         1853 non-null object
postal_code     1853 non-null object
latitude        1853 non-null float64
longitude       1853 non-null float64
stars           1853 non-null float64
review_count    1853 non-null int64
attributes      1796 non-null object
categories      1853 non-null object
dtypes: float64(3), int64(1), object(5)
memory usage: 144.8+ KB


In [8]:
df_business.head()

,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,Totum Life Science,"445 King Street W, Suite 101",M5V 1K4,43.645041,-79.395799,4.0,23,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...","Trainers, Health & Medical, Active Life, Physi..."
12,Ume Fashion Sushi,1732 Kingston Road,M1N 1S8,43.692791,-79.262630,4.5,25,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Sushi Bars, Restaurants"
15,Ravi Soups,322 Adelaide Street W,M5V 1R1,43.647585,-79.392032,4.5,332,"{'Alcohol': 'u'beer_and_wine'', 'BikeParking':...","Soup, Restaurants, Sandwiches"
17,Caffeine,1448 Lawrence Avenue E,M4A 2S6,43.742555,-79.314252,4.5,30,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Sandwiches, F..."
61,Levetto,940 College Street,M6H 1A5,43.653681,-79.426590,4.0,60,"{'GoodForMeal': '{'dessert': True, 'latenight'...","Salad, Italian, Restaurants, Pizza"


In [9]:
df_business = df_business[df_business['categories'].notna()]

In [10]:
df_business[df_business['categories'].isna()]

,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories


In [11]:
df_business.head()

,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,Totum Life Science,"445 King Street W, Suite 101",M5V 1K4,43.645041,-79.395799,4.0,23,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...","Trainers, Health & Medical, Active Life, Physi..."
12,Ume Fashion Sushi,1732 Kingston Road,M1N 1S8,43.692791,-79.262630,4.5,25,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Sushi Bars, Restaurants"
15,Ravi Soups,322 Adelaide Street W,M5V 1R1,43.647585,-79.392032,4.5,332,"{'Alcohol': 'u'beer_and_wine'', 'BikeParking':...","Soup, Restaurants, Sandwiches"
17,Caffeine,1448 Lawrence Avenue E,M4A 2S6,43.742555,-79.314252,4.5,30,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Sandwiches, F..."
61,Levetto,940 College Street,M6H 1A5,43.653681,-79.426590,4.0,60,"{'GoodForMeal': '{'dessert': True, 'latenight'...","Salad, Italian, Restaurants, Pizza"


In [12]:
df_business['duplicate_categories'] = df_business['categories'].apply(lambda row: 
                                                                        ', '.join(set(my_business['categories'].split(', '))
                                                                        .intersection(
                                                                            set(row.split(', '))
                                                                        ) ))

In [13]:
df_business.head()

,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories,duplicate_categories
0,Totum Life Science,"445 King Street W, Suite 101",M5V 1K4,43.645041,-79.395799,4.0,23,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...","Trainers, Health & Medical, Active Life, Physi...",
12,Ume Fashion Sushi,1732 Kingston Road,M1N 1S8,43.692791,-79.262630,4.5,25,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Sushi Bars, Restaurants",Restaurants
15,Ravi Soups,322 Adelaide Street W,M5V 1R1,43.647585,-79.392032,4.5,332,"{'Alcohol': 'u'beer_and_wine'', 'BikeParking':...","Soup, Restaurants, Sandwiches",Restaurants
17,Caffeine,1448 Lawrence Avenue E,M4A 2S6,43.742555,-79.314252,4.5,30,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Sandwiches, F...",Restaurants
61,Levetto,940 College Street,M6H 1A5,43.653681,-79.426590,4.0,60,"{'GoodForMeal': '{'dessert': True, 'latenight'...","Salad, Italian, Restaurants, Pizza",Restaurants


In [14]:
df_business = df_business[df_business['duplicate_categories'] != ''].reset_index()

In [15]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244 entries, 0 to 1243
Data columns (total 11 columns):
index                   1244 non-null int64
name                    1244 non-null object
address                 1244 non-null object
postal_code             1244 non-null object
latitude                1244 non-null float64
longitude               1244 non-null float64
stars                   1244 non-null float64
review_count            1244 non-null int64
attributes              1237 non-null object
categories              1244 non-null object
duplicate_categories    1244 non-null object
dtypes: float64(3), int64(2), object(6)
memory usage: 107.0+ KB


In [16]:
df_business.head()

,index,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories,duplicate_categories
0,12,Ume Fashion Sushi,1732 Kingston Road,M1N 1S8,43.692791,-79.262630,4.5,25,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Sushi Bars, Restaurants",Restaurants
1,15,Ravi Soups,322 Adelaide Street W,M5V 1R1,43.647585,-79.392032,4.5,332,"{'Alcohol': 'u'beer_and_wine'', 'BikeParking':...","Soup, Restaurants, Sandwiches",Restaurants
2,17,Caffeine,1448 Lawrence Avenue E,M4A 2S6,43.742555,-79.314252,4.5,30,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Sandwiches, F...",Restaurants
3,61,Levetto,940 College Street,M6H 1A5,43.653681,-79.426590,4.0,60,"{'GoodForMeal': '{'dessert': True, 'latenight'...","Salad, Italian, Restaurants, Pizza",Restaurants
4,68,Brockton Haunt,1150 College Street,M6H 1B6,43.652361,-79.433924,4.0,25,"{'BusinessParking': '{'garage': False, 'street...","Juice Bars & Smoothies, Restaurants, Sandwiche...",Restaurants


In [17]:
df_business['n_dup_cat'] = df_business['duplicate_categories'].apply(lambda x: len(x.split(', ')))

In [18]:
df_business['similar_%'] = df_business['n_dup_cat'] / len(set(my_business['categories'].split(', ')))

In [19]:
df_business.sort_values(['n_dup_cat', 'stars'], ascending=False).head(10)

,index,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories,duplicate_categories,n_dup_cat,similar_%
838,12279,The Haam,342 Adelaide St W,M5V 1R7,43.647387,-79.393122,4.0,83,"{'RestaurantsDelivery': 'True', 'BusinessParki...","Restaurants, Bars, Breakfast & Brunch, Sushi B...","Japanese, Nightlife, Restaurants, Asian Fusion...",6,0.857143
14,173,Odd Seoul,90 Ossington Avenue,M6J 2Y7,43.646190,-79.419686,4.0,267,"{'Alcohol': 'u'full_bar'', 'Caters': 'False', ...","Bars, Dive Bars, Korean, Restaurants, Asian Fu...","Nightlife, Restaurants, Asian Fusion, Bars, Ko...",5,0.714286
342,4725,DOMA,50C Clinton Street,M6G 2Y3,43.655522,-79.414159,4.0,58,"{'RestaurantsTakeOut': 'False', 'Smoking': 'u'...","Cocktail Bars, Bars, Korean, Restaurants, Nigh...","Nightlife, Restaurants, Asian Fusion, Bars, Ko...",5,0.714286
910,13393,Film Cafe by Eative,230 Augusta Avenue,M5T 2L6,43.655074,-79.402446,4.0,101,"{'BusinessParking': '{'garage': False, 'street...","Breakfast & Brunch, Asian Fusion, Beer Bar, Ni...","Japanese, Nightlife, Restaurants, Asian Fusion...",5,0.714286
448,6324,Lake Inez,1471 Gerrard Street E,M4L 2A1,43.672387,-79.320682,4.5,65,"{'Caters': 'False', 'BestNights': '{'monday': ...","Bars, Restaurants, Asian Fusion, Nightlife","Asian Fusion, Bars, Nightlife, Restaurants",4,0.571429
708,10401,Kevin's Taiyaki,675 Bloor Street,M6G 4B9,43.663803,-79.416272,4.5,56,"{'RestaurantsDelivery': 'False', 'RestaurantsP...","Food, Japanese, Restaurants, Asian Fusion, Kor...","Asian Fusion, Japanese, Korean, Restaurants",4,0.571429
260,3628,Ehwa J Bar,16 Isabella Street,M4Y 2A1,43.667825,-79.385215,4.0,39,"{'Ambience': '{'touristy': False, 'hipster': F...","Pubs, Korean, Nightlife, Bars, Restaurants","Korean, Bars, Nightlife, Restaurants",4,0.571429
307,4193,Mymy Chicken,9 Spring Garden Avenue,M2N,43.764280,-79.410919,4.0,161,"{'BikeParking': 'True', 'WiFi': ''no'', 'Ambie...","Bars, Korean, Nightlife, Beer Bar, Restaurants...","Korean, Bars, Nightlife, Restaurants",4,0.571429
336,4667,DaiKo Indian Kitchen & Nepali Street Food,1564 Queen Street W,M6R 1A6,43.639963,-79.440328,4.0,41,"{'RestaurantsTakeOut': 'False', 'Caters': 'Fal...","Indian, Restaurants, Beer, Wine & Spirits, Art...","Asian Fusion, Bars, Nightlife, Restaurants",4,0.571429
390,5494,Nomé Izakaya,4848 Yonge Street,M2N 5N2,43.762650,-79.411469,4.0,512,"{'HasTV': 'True', 'Ambience': '{'romantic': Fa...","Tapas Bars, Japanese, Restaurants, Lounges, Ba...","Bars, Japanese, Nightlife, Restaurants",4,0.571429


In [20]:
df_similar_business = df_business[df_business['similar_%'] > 0.5]

In [21]:
df_similar_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 14 to 1163
Data columns (total 13 columns):
index                   21 non-null int64
name                    21 non-null object
address                 21 non-null object
postal_code             21 non-null object
latitude                21 non-null float64
longitude               21 non-null float64
stars                   21 non-null float64
review_count            21 non-null int64
attributes              21 non-null object
categories              21 non-null object
duplicate_categories    21 non-null object
n_dup_cat               21 non-null int64
similar_%               21 non-null float64
dtypes: float64(4), int64(3), object(6)
memory usage: 2.3+ KB


In [22]:
df_similar_business

,index,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories,duplicate_categories,n_dup_cat,similar_%
14,173,Odd Seoul,90 Ossington Avenue,M6J 2Y7,43.646190,-79.419686,4.0,267,"{'Alcohol': 'u'full_bar'', 'Caters': 'False', ...","Bars, Dive Bars, Korean, Restaurants, Asian Fu...","Nightlife, Restaurants, Asian Fusion, Bars, Ko...",5,0.714286
260,3628,Ehwa J Bar,16 Isabella Street,M4Y 2A1,43.667825,-79.385215,4.0,39,"{'Ambience': '{'touristy': False, 'hipster': F...","Pubs, Korean, Nightlife, Bars, Restaurants","Korean, Bars, Nightlife, Restaurants",4,0.571429
307,4193,Mymy Chicken,9 Spring Garden Avenue,M2N,43.764280,-79.410919,4.0,161,"{'BikeParking': 'True', 'WiFi': ''no'', 'Ambie...","Bars, Korean, Nightlife, Beer Bar, Restaurants...","Korean, Bars, Nightlife, Restaurants",4,0.571429
336,4667,DaiKo Indian Kitchen & Nepali Street Food,1564 Queen Street W,M6R 1A6,43.639963,-79.440328,4.0,41,"{'RestaurantsTakeOut': 'False', 'Caters': 'Fal...","Indian, Restaurants, Beer, Wine & Spirits, Art...","Asian Fusion, Bars, Nightlife, Restaurants",4,0.571429
342,4725,DOMA,50C Clinton Street,M6G 2Y3,43.655522,-79.414159,4.0,58,"{'RestaurantsTakeOut': 'False', 'Smoking': 'u'...","Cocktail Bars, Bars, Korean, Restaurants, Nigh...","Nightlife, Restaurants, Asian Fusion, Bars, Ko...",5,0.714286
390,5494,Nomé Izakaya,4848 Yonge Street,M2N 5N2,43.762650,-79.411469,4.0,512,"{'HasTV': 'True', 'Ambience': '{'romantic': Fa...","Tapas Bars, Japanese, Restaurants, Lounges, Ba...","Bars, Japanese, Nightlife, Restaurants",4,0.571429
419,5773,Kintaro Izakaya,459 Church Street,M4Y 4C5,43.664179,-79.380107,4.0,134,"{'BikeParking': 'True', 'GoodForKids': 'False'...","Restaurants, Sushi Bars, Nightlife, Tapas/Smal...","Bars, Japanese, Nightlife, Restaurants",4,0.571429
420,5781,KINKA IZAKAYA BLOOR,559 Bloor St W,M5S 1Y6,43.665157,-79.410658,4.0,419,"{'BestNights': '{'monday': False, 'tuesday': F...","Tapas Bars, Pubs, Japanese, Tapas/Small Plates...","Bars, Japanese, Nightlife, Restaurants",4,0.571429
448,6324,Lake Inez,1471 Gerrard Street E,M4L 2A1,43.672387,-79.320682,4.5,65,"{'Caters': 'False', 'BestNights': '{'monday': ...","Bars, Restaurants, Asian Fusion, Nightlife","Asian Fusion, Bars, Nightlife, Restaurants",4,0.571429
614,8879,LoPan,503 College Street,M6G 1A5,43.655849,-79.409872,4.0,22,"{'BestNights': '{'monday': False, 'tuesday': F...","Bars, Nightlife, Chinese, Restaurants, Cocktai...","Asian Fusion, Bars, Nightlife, Restaurants",4,0.571429


## Find Common Attributes of Role Models

In [23]:
def adjust_quotations(attr):
    for k, v in attr.items():
        if v[0] == "{":
            v = v.replace("False", "'False'")
            v = v.replace("True", "'True'")
            v = v.replace("'", '"')
            attr[k] = json.loads(v)
        elif v[0:2] == "u'":
            v = v[2:]
            v = v[:-1]
            attr[k] = v
        else:
            v = v.replace("'", '')
            v = v.replace('"', '')            
            attr[k] = v

    return attr
            
# cleaned_my_attributes = adjust_quotations(my_business['attributes'])
# cleaned_my_attributes

In [24]:
def travel_dic(dic):
    result_list = []
    for k, v in dic.items():
        if isinstance(v, dict):
            rec_result = travel_dic(v)
            attach_key = list(map(lambda x: "{}.{}".format(k, x), rec_result))
            result_list.extend(attach_key)
        else:
            result_list.append("{}.{}".format(k, v))    
    return result_list


In [25]:
def count_dic(lst_attr):
    dic_count = {}
    for attr in lst_attr:
        if attr not in dic_count:
            dic_count[attr] = 1
        else:
            dic_count[attr] += 1
    return dic_count

In [26]:
#test
cleaned_my_attributes = adjust_quotations(my_business['attributes'])
travel = travel_dic(cleaned_my_attributes)
count_dic(travel)

{'GoodForDancing.False': 1,
 'HasTV.True': 1,
 'CoatCheck.False': 1,
 'RestaurantsPriceRange2.2': 1,
 'Smoking.no': 1,
 'RestaurantsGoodForGroups.True': 1,
 'GoodForKids.False': 1,
 'RestaurantsTakeOut.True': 1,
 'BikeParking.True': 1,
 'RestaurantsDelivery.False': 1,
 'BestNights.monday.False': 1,
 'BestNights.tuesday.False': 1,
 'BestNights.friday.False': 1,
 'BestNights.wednesday.False': 1,
 'BestNights.thursday.False': 1,
 'BestNights.sunday.False': 1,
 'BestNights.saturday.True': 1,
 'RestaurantsReservations.True': 1,
 'GoodForMeal.dessert.False': 1,
 'GoodForMeal.latenight.True': 1,
 'GoodForMeal.lunch.True': 1,
 'GoodForMeal.dinner.True': 1,
 'GoodForMeal.brunch.False': 1,
 'GoodForMeal.breakfast.False': 1,
 'BusinessParking.garage.False': 1,
 'BusinessParking.street.True': 1,
 'BusinessParking.validated.False': 1,
 'BusinessParking.lot.False': 1,
 'BusinessParking.valet.False': 1,
 'HappyHour.False': 1,
 'Caters.False': 1,
 'RestaurantsTableService.True': 1,
 'OutdoorSeating.Fa

In [27]:
def check_all_restaurants_attributes(df_column):
    final_count = {}
    for attr in df_column:
        cleaned = adjust_quotations(attr)
        travel = travel_dic(cleaned)
        local_count = count_dic(travel)
        for k, v in local_count.items():
            if k not in final_count:
                final_count[k] = v
            else:
                final_count[k] += v
    for k, v in final_count.items():
        final_count[k] = "{0:.2f}".format(round(v / df_column.size,2))
    return final_count

In [28]:
count = check_all_restaurants_attributes(df_similar_business['attributes'])
common_attributes = sorted(count.items(), key=lambda kv: kv[1], reverse=True)

In [29]:
common_attributes

[('BusinessParking.garage.False', '1.00'),
 ('BusinessParking.lot.False', '1.00'),
 ('BusinessParking.valet.False', '1.00'),
 ('Ambience.touristy.False', '0.95'),
 ('Ambience.upscale.False', '0.95'),
 ('Ambience.classy.False', '0.95'),
 ('BusinessParking.validated.False', '0.95'),
 ('GoodForMeal.dessert.False', '0.90'),
 ('Ambience.romantic.False', '0.90'),
 ('RestaurantsAttire.casual', '0.86'),
 ('RestaurantsDelivery.False', '0.86'),
 ('GoodForMeal.brunch.False', '0.86'),
 ('GoodForMeal.breakfast.False', '0.86'),
 ('Ambience.intimate.False', '0.86'),
 ('Alcohol.full_bar', '0.81'),
 ('RestaurantsGoodForGroups.True', '0.81'),
 ('GoodForMeal.latenight.False', '0.81'),
 ('GoodForMeal.lunch.False', '0.81'),
 ('Ambience.hipster.False', '0.81'),
 ('BikeParking.True', '0.76'),
 ('BusinessParking.street.True', '0.76'),
 ('RestaurantsTakeOut.True', '0.71'),
 ('HasTV.False', '0.67'),
 ('Ambience.trendy.False', '0.67'),
 ('GoodForKids.False', '0.62'),
 ('GoodForMeal.dinner.True', '0.62'),
 ('Rest